## Day 13

In [1]:
import os
import numpy as np

def read_file(filename="input.txt", path=os.path.join(os.getcwd())):
    with open(os.path.join(path, filename), "r") as f:
        input_list = [line.strip() for line in f]
    return input_list

In [5]:
example = [
    "6,10",
    "0,14",
    "9,10",
    "0,3",
    "10,4",
    "4,11",
    "6,0",
    "6,12",
    "4,1",
    "0,13",
    "10,12",
    "3,4",
    "3,0",
    "8,4",
    "1,10",
    "2,14",
    "8,10",
    "9,0",
    "",
    "fold along y=7",
    "fold along x=5",
]

In [27]:
def preprocess_input(input_list):
    dots = []
    fold_instruction = False
    fold_instructions = []
    for line in input_list:        
        if line:
            if fold_instruction:
                i, j = line.split('=')
                fold_instructions.append((i[-1], int(j)))
            else:
                i, j = line.split(',')
                dots.append((int(i), int(j)))
        else:
            fold_instruction = True
        
    return dots, fold_instructions

def create_paper(dots):
    dots = np.array(dots)
    
    paper = np.zeros((max(dots[:, 1])+1, max(dots[:, 0])+1), dtype=np.int)
    
    for x, y in dots:
        paper[y, x] = 1
    
    return paper

def fold_up(paper, y):
    upper_part = paper[:y, :]
    lower_part = paper[y+1:, :]
    
    if lower_part.shape[0] > upper_part.shape[0]:
        lower_part = lower_part[:upper_part.shape[0], :]
    elif lower_part.shape[0] < upper_part.shape[0]:
        diff = upper_part.shape[0] - lower_part.shape[0]
        padd = np.zeros((diff, lower_part.shape[1]))
        lower_part = np.concatenate((lower_part, padd), axis=0)
    
    lower_part = lower_part[::-1]
    
    return np.array((upper_part+lower_part) > 0, dtype=np.int)

def fold_left(paper, x):
    left_part = paper[:, :x]
    right_part = paper[:, x+1:]
    
    if right_part.shape[1] > left_part.shape[1]:
        right_part = lower_part[:, :right_part.shape[1],]
        
    elif right_part.shape[1] < left_part.shape[1]:
        diff = left_part.shape[1] - right_part.shape[1]
        padd = np.zeros((right_part.shape[0], diff))
        right_part = np.concatenate((right_part, padd), axis=1)
    
    right_part = np.array([line[::-1] for line in right_part])
    
    return np.array((left_part+right_part) > 0, dtype=np.int)

In [32]:
dots, fold_instructions = preprocess_input(read_file())
paper = create_paper(dots)

for instruction, fold in fold_instructions:
    if instruction == 'x':
        paper = fold_left(paper, fold)
    elif instruction == 'y':
        paper = fold_up(paper, fold)
    break
        
print(f"Dots after the first fold {np.count_nonzero(paper)}")

Dots after the first fold 704


In [39]:
dots, fold_instructions = preprocess_input(read_file())
paper = create_paper(dots)

for instruction, fold in fold_instructions:
    if instruction == 'x':
        paper = fold_left(paper, fold)
    elif instruction == 'y':
        paper = fold_up(paper, fold)
        

for line in paper:
    l = ""
    for n in line:
        if n == 1:
            l += "#"
        else:
            l += "."
    print(l)

#..#..##...##....##.###..####.#..#..##..
#..#.#..#.#..#....#.#..#.#....#..#.#..#.
####.#....#..#....#.###..###..####.#....
#..#.#.##.####....#.#..#.#....#..#.#....
#..#.#..#.#..#.#..#.#..#.#....#..#.#..#.
#..#..###.#..#..##..###..####.#..#..##..
